# Pytorch构建神经网络(三)(29-33节)

## 4.1&4.2 使用tensorboard可视化CNN训练指标

* pytorch1.1.0以上的版本已经自动增加了tensorboard
* 在终端输入“tensorboard --version”可查看tensorboard的版本
* 在终端输入“tensorboard --logdir=runs”进入tensorboard(在写了tensorboard数据的路径下)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)


In [2]:
print(torch.__version__)
print(torchvision.__version__)

2.1.0.dev20230618+cu121
0.16.0.dev20230619+cu121


In [3]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        t = t 
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.reshape(-1, 12*4*4)  # t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        return t
        

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)
#train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

### Starting out with TensorBoard (Network Graph and Images)

In [6]:
#batch_size = 100
#lr =0.01
# 对不同的batchsize，lr的训练情况进行比较
# 方法1：但此方法需要多层for循环
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]
for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)   # 创建能在tensorboard中查看的图像网格

        comment = f'batch_size={batch_size} lr ={lr}'
        tb = SummaryWriter(comment=comment)   # 在Summary Writer添加该注释，可帮助我们在tensorboard中唯一地识别该表示
        tb.add_image('images', grid)  # 将一批图像放在grid中进行显示
        tb.add_graph(network, images)   # 在tensorboard中看见网络结构的可视化图
        optimizer = optim.Adam(network.parameters(), lr=lr)

        for epoch in range(5):
    
            total_loss = 0
            total_correct = 0
    
            for batch in train_loader:    # Get Batch
                images, labels = batch
        
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels)  # Calculate loss
        
                optimizer.zero_grad()    # 梯度清零，否则会累加
                loss.backward()     # Calculate Gradients
                optimizer.step()    # Update Weights
        
                #total_loss += loss.item()
                total_loss += loss.item()*batch_size # 在对不同批次下的训练进行比较时，这样做可使结果更具有可比性
                total_correct += get_num_correct(preds, labels)
        
            tb.add_scalar("Loss", total_loss, epoch)
            tb.add_scalar("Number Correct", total_correct, epoch)
            tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)
            '''
            这种表达方式只能看单个层的偏置，权重，及其梯度的变化趋势，无法看到全部的
            tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
            tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
            tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
            '''
            for name, weight in network.named_parameters():
                tb.add_histogram(name, weight, epoch)
                tb.add_histogram(f'{name}.grad', weight.grad, epoch)
            print("epoch:", epoch, "total_correct:", total_correct, "loss", total_loss)

tb.close()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


epoch: 0 total_correct: 46050 loss 36506.88832998276
epoch: 1 total_correct: 51128 loss 23707.560509443283
epoch: 2 total_correct: 51959 loss 21809.392841160297
epoch: 3 total_correct: 52445 loss 20498.195345699787
epoch: 4 total_correct: 52569 loss 20044.924208521843
epoch: 0 total_correct: 41278 loss 49702.53894031048
epoch: 1 total_correct: 48206 loss 31598.01917076111
epoch: 2 total_correct: 50467 loss 26478.306898474693
epoch: 3 total_correct: 51441 loss 23607.636871933937
epoch: 4 total_correct: 52068 loss 21886.25321686268
epoch: 0 total_correct: 31750 loss 82063.28451633453
epoch: 1 total_correct: 42110 loss 47667.15636253357
epoch: 2 total_correct: 43970 loss 42805.135479569435
epoch: 3 total_correct: 45042 loss 39903.14244925976
epoch: 4 total_correct: 45885 loss 37850.90346634388
epoch: 0 total_correct: 6000 loss 138115.762257576
epoch: 1 total_correct: 11515 loss 134527.77144908905
epoch: 2 total_correct: 21393 loss 120349.46969747543
epoch: 3 total_correct: 30955 loss 9870

#### 对多层的偏置，权重及其梯度进行访问的原理

In [7]:
for name,weight in network.named_parameters():
    print(name, weight.shape)

conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([12, 6, 5, 5])
conv2.bias torch.Size([12])
fc1.weight torch.Size([120, 192])
fc1.bias torch.Size([120])
fc2.weight torch.Size([60, 120])
fc2.bias torch.Size([60])
out.weight torch.Size([10, 60])
out.bias torch.Size([10])


In [8]:
for name,weight in network.named_parameters():
    print(f'{name}.grad', weight.grad.shape)

conv1.weight.grad torch.Size([6, 1, 5, 5])
conv1.bias.grad torch.Size([6])
conv2.weight.grad torch.Size([12, 6, 5, 5])
conv2.bias.grad torch.Size([12])
fc1.weight.grad torch.Size([120, 192])
fc1.bias.grad torch.Size([120])
fc2.weight.grad torch.Size([60, 120])
fc2.bias.grad torch.Size([60])
out.weight.grad torch.Size([10, 60])
out.bias.grad torch.Size([10])


#### 更简单的方法对要更改的参数进行访问

In [9]:
from itertools import product

In [10]:
parameters = dict(
    lr = [.01, .001],
    batc_size = [10, 100, 1000],
    shuffle = [True, False]
)

In [11]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

In [12]:
for lr, batch_size, shuffle in product(*param_values):
    print(lr, batch_size, shuffle)

0.01 10 True
0.01 10 False
0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 10 True
0.001 10 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [13]:
#batch_size = 100
#lr =0.01
# 对不同的batchsize，lr的训练情况进行比较
# 方法2：只需一层循环
from itertools import product
parameters = dict(
    lr = [.01, .001],
    batch_size = [10, 100],
    shuffle = [True, False]
)
param_values = [v for v in parameters.values()]
print(param_values)
for lr, batch_size, shuffle in product(*param_values):
    network = Network()
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    images, labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)   # 创建能在tensorboard中查看的图像网格

    comment = f'batch_size={batch_size} lr ={lr} shuffle={shuffle}'
    tb = SummaryWriter(comment=comment)   # 在Summary Writer添加该注释，可帮助我们在tensorboard中唯一地识别该表示
    tb.add_image('images', grid)  # 将一批图像放在grid中进行显示
    tb.add_graph(network, images)   # 在tensorboard中看见网络结构的可视化图
    optimizer = optim.Adam(network.parameters(), lr=lr)

    for epoch in range(5):
    
        total_loss = 0
        total_correct = 0
    
        for batch in train_loader:    # Get Batch
            images, labels = batch
        
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels)  # Calculate loss
        
            optimizer.zero_grad()    # 梯度清零，否则会累加
            loss.backward()     # Calculate Gradients
            optimizer.step()    # Update Weights
        
            #total_loss += loss.item()
            total_loss += loss.item()*batch_size # 在对不同批次下的训练进行比较时，这样做可使结果更具有可比性
            total_correct += get_num_correct(preds, labels)
        
        tb.add_scalar("Loss", total_loss, epoch)
        tb.add_scalar("Number Correct", total_correct, epoch)
        tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)
        '''
            这种表达方式只能看单个层的偏置，权重，及其梯度的变化趋势，无法看到全部的
            tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
            tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
            tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
        '''
        for name, weight in network.named_parameters():  # loop all the name to access all the layers
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f'{name}.grad', weight.grad, epoch)
        print("epoch:", epoch, "total_correct:", total_correct, "loss", total_loss)

tb.close()

[[0.01, 0.001], [10, 100], [True, False]]
epoch: 0 total_correct: 46361 loss 36430.55327379145
epoch: 1 total_correct: 48245 loss 32111.23051509261
epoch: 2 total_correct: 48607 loss 31409.231896260753
epoch: 3 total_correct: 48331 loss 32717.400109448936
epoch: 4 total_correct: 47905 loss 33666.40711192973
epoch: 0 total_correct: 46760 loss 35684.513980615884
epoch: 1 total_correct: 48798 loss 31283.186269183643
epoch: 2 total_correct: 49227 loss 30276.362059894018
epoch: 3 total_correct: 49498 loss 29886.548443087377
epoch: 4 total_correct: 49471 loss 29694.766083842842
epoch: 0 total_correct: 46513 loss 35256.35792315006
epoch: 1 total_correct: 50945 loss 24265.49262702465
epoch: 2 total_correct: 51925 loss 21870.929792523384
epoch: 3 total_correct: 52278 loss 20779.40961867571
epoch: 4 total_correct: 52595 loss 20128.92806082964
epoch: 0 total_correct: 46668 loss 34880.756129324436
epoch: 1 total_correct: 51174 loss 23617.76382178068
epoch: 2 total_correct: 51902 loss 21649.5079904

## 4.3 RunBuilder类的编写
* 该类的编写允许我们使用不同的参数值生成多个运行

In [14]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [15]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [16]:
params = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000]
)

In [17]:
runs = RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [18]:
for run in runs:
    print(run, run.lr, run.batch_size)

Run(lr=0.01, batch_size=1000) 0.01 1000
Run(lr=0.01, batch_size=10000) 0.01 10000
Run(lr=0.001, batch_size=1000) 0.001 1000
Run(lr=0.001, batch_size=10000) 0.001 10000


In [19]:
# 创建RunBuilder类以后，comment表示为：
for run in RunBuilder.get_runs(params):
    comment = f'-{run}'
    print(comment)

-Run(lr=0.01, batch_size=1000)
-Run(lr=0.01, batch_size=10000)
-Run(lr=0.001, batch_size=1000)
-Run(lr=0.001, batch_size=10000)


# 4.4 如何试验大量的超参数
* 构建RunManager类可实现对大量超参数的试验

In [26]:
import time
import pandas as pd
import json

class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run, network, loader):
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('images', grid)
        self.tb.add_graph(self.network, images)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
    
    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
            
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        #clear_output(wait=True)  # jupyter command
        display(df)
    
    def track_loss(self, loss):
        self.epoch_loss += loss.item()*self.loader.batch_size
    
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(f'{fileName}.csv')
        with open(f'{fileName},json','w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [27]:
# 使用RunManager和RunBuilder类可以使得程序更易扩展
params = OrderedDict(
    lr = [.01],
    batch_size =[1000, 2000],
    shuffle = [True, False]
)
m = RunManager()
for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = torch.utils.data.DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
            
        m.end_epoch()
    m.end_run()
m.save('resuls')

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.047064,0.6024,8.56721,9.126302,0.01,1000,True


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.047064,0.602400,8.567210,9.126302,0.01,1000,True
1,1,2,0.539757,0.794283,8.326803,17.516111,0.01,1000,True


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


...

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.047064,0.602400,8.567210,9.126302,0.01,1000,True
1,1,2,0.539757,0.794283,8.326803,17.516111,0.01,1000,True
2,1,3,0.439877,0.837167,8.366158,25.928188,0.01,1000,True
3,1,4,0.385829,0.858367,8.725152,34.697342,0.01,1000,True
4,1,5,0.355282,0.869433,9.143626,43.892480,0.01,1000,True
5,2,1,0.901662,0.654883,11.421416,12.040163,0.01,1000,False
6,2,2,0.489757,0.811500,10.372510,22.464417,0.01,1000,False
7,2,3,0.399005,0.852783,10.162603,32.671983,0.01,1000,False
8,2,4,0.356661,0.869150,9.919597,42.639706,0.01,1000,False
9,2,5,0.328820,0.879267,10.010163,52.693855,0.01,1000,False


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.047064,0.602400,8.567210,9.126302,0.01,1000,True
1,1,2,0.539757,0.794283,8.326803,17.516111,0.01,1000,True
2,1,3,0.439877,0.837167,8.366158,25.928188,0.01,1000,True
3,1,4,0.385829,0.858367,8.725152,34.697342,0.01,1000,True
4,1,5,0.355282,0.869433,9.143626,43.892480,0.01,1000,True
5,2,1,0.901662,0.654883,11.421416,12.040163,0.01,1000,False
6,2,2,0.489757,0.811500,10.372510,22.464417,0.01,1000,False
7,2,3,0.399005,0.852783,10.162603,32.671983,0.01,1000,False
8,2,4,0.356661,0.869150,9.919597,42.639706,0.01,1000,False
9,2,5,0.328820,0.879267,10.010163,52.693855,0.01,1000,False


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.047064,0.602400,8.567210,9.126302,0.01,1000,True
1,1,2,0.539757,0.794283,8.326803,17.516111,0.01,1000,True
2,1,3,0.439877,0.837167,8.366158,25.928188,0.01,1000,True
3,1,4,0.385829,0.858367,8.725152,34.697342,0.01,1000,True
4,1,5,0.355282,0.869433,9.143626,43.892480,0.01,1000,True
5,2,1,0.901662,0.654883,11.421416,12.040163,0.01,1000,False
6,2,2,0.489757,0.811500,10.372510,22.464417,0.01,1000,False
7,2,3,0.399005,0.852783,10.162603,32.671983,0.01,1000,False
8,2,4,0.356661,0.869150,9.919597,42.639706,0.01,1000,False
9,2,5,0.328820,0.879267,10.010163,52.693855,0.01,1000,False


# 4.5 使用DataLoader的多进程功能加速神经网络训练

* 使用data loader类的num_workers可选属性可加速神经网络的训练
* num_workers属性告诉data loader实例有多少个单元处理器用于数据加载
* num_workers值的选择的最好方式是进行试验

In [28]:
# 使用RunManager和RunBuilder类可以使得程序更易扩展
params = OrderedDict(
    lr = [.01],
    batch_size =[1000, 2000],
    shuffle = [True, False],
    num_workers = [0,1,2,4,8,16] ) 
m = RunManager()
for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = torch.utils.data.DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
            
            
        m.end_epoch()
    m.end_run()
m.save('resuls')

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,1.008196,0.61105,9.4461,10.59409,0.01,1000,True,0


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,1.008196,0.61105,9.446100,10.594090,0.01,1000,True,0
1,1,2,0.512543,0.80390,8.590305,19.226487,0.01,1000,True,0


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


...

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,1.008196,0.611050,9.446100,10.594090,0.01,1000,True,0
1,1,2,0.512543,0.803900,8.590305,19.226487,0.01,1000,True,0
2,1,3,0.412982,0.848717,8.652989,27.928881,0.01,1000,True,0
3,1,4,0.370922,0.862800,8.747153,36.718138,0.01,1000,True,0
4,1,5,0.341136,0.872867,8.601278,45.364862,0.01,1000,True,0
...,...,...,...,...,...,...,...,...,...,...
113,23,4,0.459227,0.831417,9.381711,43.281609,0.01,2000,False,8
114,23,5,0.419026,0.847267,9.011104,52.333362,0.01,2000,False,8
115,24,1,1.192625,0.564400,12.208997,21.887348,0.01,2000,False,16
116,24,2,0.591575,0.768817,12.463262,34.398143,0.01,2000,False,16


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,1.008196,0.611050,9.446100,10.594090,0.01,1000,True,0
1,1,2,0.512543,0.803900,8.590305,19.226487,0.01,1000,True,0
2,1,3,0.412982,0.848717,8.652989,27.928881,0.01,1000,True,0
3,1,4,0.370922,0.862800,8.747153,36.718138,0.01,1000,True,0
4,1,5,0.341136,0.872867,8.601278,45.364862,0.01,1000,True,0
...,...,...,...,...,...,...,...,...,...,...
114,23,5,0.419026,0.847267,9.011104,52.333362,0.01,2000,False,8
115,24,1,1.192625,0.564400,12.208997,21.887348,0.01,2000,False,16
116,24,2,0.591575,0.768817,12.463262,34.398143,0.01,2000,False,16
117,24,3,0.485875,0.809500,12.743652,47.191311,0.01,2000,False,16


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,1.008196,0.611050,9.446100,10.594090,0.01,1000,True,0
1,1,2,0.512543,0.803900,8.590305,19.226487,0.01,1000,True,0
2,1,3,0.412982,0.848717,8.652989,27.928881,0.01,1000,True,0
3,1,4,0.370922,0.862800,8.747153,36.718138,0.01,1000,True,0
4,1,5,0.341136,0.872867,8.601278,45.364862,0.01,1000,True,0
...,...,...,...,...,...,...,...,...,...,...
115,24,1,1.192625,0.564400,12.208997,21.887348,0.01,2000,False,16
116,24,2,0.591575,0.768817,12.463262,34.398143,0.01,2000,False,16
117,24,3,0.485875,0.809500,12.743652,47.191311,0.01,2000,False,16
118,24,4,0.436188,0.834300,12.477635,59.717745,0.01,2000,False,16
